In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
# open text file and read in data as `text`
with open('C:/Users/TimurRashitov/Desktop/sdg.txt', 'r') as f:
    text = f.read()

In [3]:
text[:100]

'“Pure intellectual stimulation that can be popped into\nthe [audio or video player] anytime.”\n—Harvar'

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
len(char2int)

106

In [6]:
encoded.max()

105

In [7]:
encoded[0:100]

array([26, 30,  6, 98, 31, 33, 67,  7, 78, 31, 96, 96, 31, 92, 78,  6, 56,
       96, 33, 27, 78, 67, 34,  6, 96, 56, 78, 67, 49,  7, 33, 78, 90, 56,
       78, 33, 92, 56,  7, 33, 40, 31, 33, 47, 49, 47, 47, 31,  1, 33, 67,
        7, 78, 49, 87, 78, 90, 31, 33, 24, 56,  6,  1, 67, 49, 33, 49, 98,
       33, 64, 67,  1, 31, 49, 33, 47, 96, 56, 59, 31, 98, 20, 33, 56,  7,
       59, 78, 67, 34, 31, 45, 63, 87, 77, 18, 56, 98, 64, 56, 98])

In [8]:
for i in encoded[0:10]:
    print(int2char[i])

“
P
u
r
e
 
i
n
t
e


In [9]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [10]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1],
                    [1, 1, 1],
                   [3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [11]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [12]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [13]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[26 30  6 98 31 33 67  7 78 31]
 [98 31  7 78 45 33 12  7 33 78]
 [96 33 98 31 96 31  7 78 96 31]
 [49  7  7 31 92 78 27 33 78 90]
 [38 49 33 92 56 96 96 31 98 27]
 [34 87  8 40 56  7 21 56 33 65]
 [ 6  7 33 78 49 33  1 31 64 31]
 [98 93  6  7  1 67 56  7 33 96]]

y
 [[30  6 98 31 33 67  7 78 31 96]
 [31  7 78 45 33 12  7 33 78 90]
 [33 98 31 96 31  7 78 96 31 27]
 [ 7  7 31 92 78 27 33 78 90 31]
 [49 33 92 56 96 96 31 98 27 33]
 [87  8 40 56  7 21 56 33 65 49]
 [ 7 33 78 49 33  1 31 64 31 96]
 [93  6  7  1 67 56  7 33 96 31]]


In [14]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [15]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [16]:
def train(net, data, epochs=100, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=1):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    valid_loss_min = np.Inf
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            valid_loss_ = 0    
            
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                valid_loss_ = np.mean(val_losses)
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(valid_loss_))
                
                if valid_loss_ <= valid_loss_min:
                    print('Valid metrics improved ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss_))
                    model_name = 'text_gen.net'
                    checkpoint = {'n_hidden': net.n_hidden,
                                  'n_layers': net.n_layers,
                                  'state_dict': net.state_dict(),
                                  'tokens': net.chars}
                    with open(model_name, 'wb') as f:
                        torch.save(checkpoint, f)
                    valid_loss_min = valid_loss_
                    print("=========================================")
                

In [17]:
# define and print the net
n_hidden=256
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(106, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=106, bias=True)
)


In [18]:
batch_size = 128
seq_length = 100
n_epochs = 50 # start smaller if you are just testing initial behavior

In [19]:
# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/50... Step: 10... Loss: 3.5995... Val Loss: 3.6462
Valid metrics improved (inf --> 3.646210).  Saving model ...
Epoch: 1/50... Step: 20... Loss: 3.4842... Val Loss: 3.5288
Valid metrics improved (3.646210 --> 3.528760).  Saving model ...
Epoch: 1/50... Step: 30... Loss: 3.4415... Val Loss: 3.5174
Valid metrics improved (3.528760 --> 3.517430).  Saving model ...
Epoch: 1/50... Step: 40... Loss: 3.3793... Val Loss: 3.5157
Valid metrics improved (3.517430 --> 3.515733).  Saving model ...
Epoch: 1/50... Step: 50... Loss: 3.3931... Val Loss: 3.5137
Valid metrics improved (3.515733 --> 3.513655).  Saving model ...
Epoch: 1/50... Step: 60... Loss: 3.4121... Val Loss: 3.5150
Epoch: 1/50... Step: 70... Loss: 3.4027... Val Loss: 3.5103
Valid metrics improved (3.513655 --> 3.510304).  Saving model ...
Epoch: 1/50... Step: 80... Loss: 3.4362... Val Loss: 3.5072
Valid metrics improved (3.510304 --> 3.507186).  Saving model ...
Epoch: 1/50... Step: 90... Loss: 3.4318... Val Loss: 3.5059
Val

Epoch: 3/50... Step: 520... Loss: 2.5438... Val Loss: 2.5519
Epoch: 3/50... Step: 530... Loss: 2.5270... Val Loss: 2.5518
Epoch: 3/50... Step: 540... Loss: 2.5261... Val Loss: 2.5299
Valid metrics improved (2.551417 --> 2.529949).  Saving model ...
Epoch: 4/50... Step: 550... Loss: 2.5151... Val Loss: 2.5251
Valid metrics improved (2.529949 --> 2.525090).  Saving model ...
Epoch: 4/50... Step: 560... Loss: 2.5426... Val Loss: 2.5173
Valid metrics improved (2.525090 --> 2.517319).  Saving model ...
Epoch: 4/50... Step: 570... Loss: 2.5083... Val Loss: 2.5058
Valid metrics improved (2.517319 --> 2.505780).  Saving model ...
Epoch: 4/50... Step: 580... Loss: 2.5130... Val Loss: 2.4985
Valid metrics improved (2.505780 --> 2.498475).  Saving model ...
Epoch: 4/50... Step: 590... Loss: 2.5098... Val Loss: 2.4950
Valid metrics improved (2.498475 --> 2.494952).  Saving model ...
Epoch: 4/50... Step: 600... Loss: 2.4846... Val Loss: 2.4901
Valid metrics improved (2.494952 --> 2.490120).  Saving

Epoch: 6/50... Step: 1040... Loss: 2.2458... Val Loss: 2.2821
Valid metrics improved (2.284544 --> 2.282120).  Saving model ...
Epoch: 6/50... Step: 1050... Loss: 2.2685... Val Loss: 2.2777
Valid metrics improved (2.282120 --> 2.277729).  Saving model ...
Epoch: 6/50... Step: 1060... Loss: 2.2692... Val Loss: 2.2725
Valid metrics improved (2.277729 --> 2.272508).  Saving model ...
Epoch: 6/50... Step: 1070... Loss: 2.2703... Val Loss: 2.2674
Valid metrics improved (2.272508 --> 2.267356).  Saving model ...
Epoch: 6/50... Step: 1080... Loss: 2.2499... Val Loss: 2.2640
Valid metrics improved (2.267356 --> 2.263965).  Saving model ...
Epoch: 7/50... Step: 1090... Loss: 2.2450... Val Loss: 2.2604
Valid metrics improved (2.263965 --> 2.260388).  Saving model ...
Epoch: 7/50... Step: 1100... Loss: 2.2780... Val Loss: 2.2571
Valid metrics improved (2.260388 --> 2.257126).  Saving model ...
Epoch: 7/50... Step: 1110... Loss: 2.2302... Val Loss: 2.2515
Valid metrics improved (2.257126 --> 2.251

Epoch: 9/50... Step: 1570... Loss: 2.1072... Val Loss: 2.1175
Valid metrics improved (2.119711 --> 2.117478).  Saving model ...
Epoch: 9/50... Step: 1580... Loss: 2.1169... Val Loss: 2.1166
Valid metrics improved (2.117478 --> 2.116555).  Saving model ...
Epoch: 9/50... Step: 1590... Loss: 2.1378... Val Loss: 2.1093
Valid metrics improved (2.116555 --> 2.109268).  Saving model ...
Epoch: 9/50... Step: 1600... Loss: 2.1065... Val Loss: 2.1082
Valid metrics improved (2.109268 --> 2.108233).  Saving model ...
Epoch: 9/50... Step: 1610... Loss: 2.0963... Val Loss: 2.1029
Valid metrics improved (2.108233 --> 2.102917).  Saving model ...
Epoch: 9/50... Step: 1620... Loss: 2.0831... Val Loss: 2.0994
Valid metrics improved (2.102917 --> 2.099397).  Saving model ...
Epoch: 10/50... Step: 1630... Loss: 2.1301... Val Loss: 2.0981
Valid metrics improved (2.099397 --> 2.098117).  Saving model ...
Epoch: 10/50... Step: 1640... Loss: 2.1050... Val Loss: 2.0977
Valid metrics improved (2.098117 --> 2.0

Epoch: 12/50... Step: 2120... Loss: 2.0152... Val Loss: 1.9923
Valid metrics improved (1.993997 --> 1.992292).  Saving model ...
Epoch: 12/50... Step: 2130... Loss: 2.0327... Val Loss: 1.9909
Valid metrics improved (1.992292 --> 1.990925).  Saving model ...
Epoch: 12/50... Step: 2140... Loss: 1.9712... Val Loss: 1.9865
Valid metrics improved (1.990925 --> 1.986473).  Saving model ...
Epoch: 12/50... Step: 2150... Loss: 1.9961... Val Loss: 1.9866
Epoch: 12/50... Step: 2160... Loss: 1.9788... Val Loss: 1.9871
Epoch: 12/50... Step: 2170... Loss: 1.9603... Val Loss: 1.9807
Valid metrics improved (1.986473 --> 1.980703).  Saving model ...
Epoch: 13/50... Step: 2180... Loss: 1.9655... Val Loss: 1.9824
Epoch: 13/50... Step: 2190... Loss: 1.9688... Val Loss: 1.9842
Epoch: 13/50... Step: 2200... Loss: 1.9754... Val Loss: 1.9778
Valid metrics improved (1.980703 --> 1.977762).  Saving model ...
Epoch: 13/50... Step: 2210... Loss: 2.0004... Val Loss: 1.9744
Valid metrics improved (1.977762 --> 1.9

...

Epoch: 28/50... Step: 5040... Loss: 1.6726... Val Loss: 1.7082
Epoch: 28/50... Step: 5050... Loss: 1.6476... Val Loss: 1.7090
Epoch: 28/50... Step: 5060... Loss: 1.6496... Val Loss: 1.7066
Valid metrics improved (1.706800 --> 1.706608).  Saving model ...
Epoch: 29/50... Step: 5070... Loss: 1.6888... Val Loss: 1.7076
Epoch: 29/50... Step: 5080... Loss: 1.6400... Val Loss: 1.7072
Epoch: 29/50... Step: 5090... Loss: 1.6169... Val Loss: 1.7065
Valid metrics improved (1.706608 --> 1.706451).  Saving model ...
Epoch: 29/50... Step: 5100... Loss: 1.6570... Val Loss: 1.7056
Valid metrics improved (1.706451 --> 1.705603).  Saving model ...
Epoch: 29/50... Step: 5110... Loss: 1.6412... Val Loss: 1.7057
Epoch: 29/50... Step: 5120... Loss: 1.6209... Val Loss: 1.7066
Epoch: 29/50... Step: 5130... Loss: 1.6699... Val Loss: 1.7038
Valid metrics improved (1.705603 --> 1.703843).  Saving model ...
Epoch: 29/50... Step: 5140... Loss: 1.6232... Val Loss: 1.7025
Valid metrics improved (1.703843 --> 1.7024

Epoch: 33/50... Step: 5920... Loss: 1.6341... Val Loss: 1.6738
Valid metrics improved (1.676758 --> 1.673771).  Saving model ...
Epoch: 33/50... Step: 5930... Loss: 1.5970... Val Loss: 1.6749
Epoch: 33/50... Step: 5940... Loss: 1.5997... Val Loss: 1.6749
Epoch: 33/50... Step: 5950... Loss: 1.6112... Val Loss: 1.6742
Epoch: 33/50... Step: 5960... Loss: 1.6127... Val Loss: 1.6724
Valid metrics improved (1.673771 --> 1.672409).  Saving model ...
Epoch: 33/50... Step: 5970... Loss: 1.6273... Val Loss: 1.6716
Valid metrics improved (1.672409 --> 1.671568).  Saving model ...
Epoch: 34/50... Step: 5980... Loss: 1.5949... Val Loss: 1.6734
Epoch: 34/50... Step: 5990... Loss: 1.6266... Val Loss: 1.6752
Epoch: 34/50... Step: 6000... Loss: 1.6008... Val Loss: 1.6733
Epoch: 34/50... Step: 6010... Loss: 1.5901... Val Loss: 1.6754
Epoch: 34/50... Step: 6020... Loss: 1.6232... Val Loss: 1.6746
Epoch: 34/50... Step: 6030... Loss: 1.5791... Val Loss: 1.6776
Epoch: 34/50... Step: 6040... Loss: 1.6232... 

Epoch: 38/50... Step: 6870... Loss: 1.5901... Val Loss: 1.6554
Epoch: 39/50... Step: 6880... Loss: 1.6217... Val Loss: 1.6535
Epoch: 39/50... Step: 6890... Loss: 1.5628... Val Loss: 1.6514
Valid metrics improved (1.653111 --> 1.651426).  Saving model ...
Epoch: 39/50... Step: 6900... Loss: 1.5347... Val Loss: 1.6529
Epoch: 39/50... Step: 6910... Loss: 1.5927... Val Loss: 1.6512
Valid metrics improved (1.651426 --> 1.651193).  Saving model ...
Epoch: 39/50... Step: 6920... Loss: 1.5673... Val Loss: 1.6524
Epoch: 39/50... Step: 6930... Loss: 1.5497... Val Loss: 1.6514
Epoch: 39/50... Step: 6940... Loss: 1.6041... Val Loss: 1.6519
Epoch: 39/50... Step: 6950... Loss: 1.5542... Val Loss: 1.6513
Epoch: 39/50... Step: 6960... Loss: 1.5773... Val Loss: 1.6485
Valid metrics improved (1.651193 --> 1.648507).  Saving model ...
Epoch: 39/50... Step: 6970... Loss: 1.6165... Val Loss: 1.6539
Epoch: 39/50... Step: 6980... Loss: 1.5355... Val Loss: 1.6516
Epoch: 39/50... Step: 6990... Loss: 1.5953... 

Epoch: 44/50... Step: 7870... Loss: 1.5508... Val Loss: 1.6362
Epoch: 44/50... Step: 7880... Loss: 1.5707... Val Loss: 1.6341
Epoch: 44/50... Step: 7890... Loss: 1.5400... Val Loss: 1.6363
Epoch: 44/50... Step: 7900... Loss: 1.5634... Val Loss: 1.6361
Epoch: 44/50... Step: 7910... Loss: 1.5590... Val Loss: 1.6366
Epoch: 44/50... Step: 7920... Loss: 1.5679... Val Loss: 1.6373
Epoch: 44/50... Step: 7930... Loss: 1.5381... Val Loss: 1.6362
Epoch: 44/50... Step: 7940... Loss: 1.5379... Val Loss: 1.6339
Epoch: 44/50... Step: 7950... Loss: 1.5395... Val Loss: 1.6302
Valid metrics improved (1.633018 --> 1.630193).  Saving model ...
Epoch: 44/50... Step: 7960... Loss: 1.5719... Val Loss: 1.6310
Epoch: 45/50... Step: 7970... Loss: 1.5593... Val Loss: 1.6329
Epoch: 45/50... Step: 7980... Loss: 1.5526... Val Loss: 1.6325
Epoch: 45/50... Step: 7990... Loss: 1.5322... Val Loss: 1.6310
Epoch: 45/50... Step: 8000... Loss: 1.5105... Val Loss: 1.6333
Epoch: 45/50... Step: 8010... Loss: 1.5446... Val Lo

Epoch: 50/50... Step: 9000... Loss: 1.5077... Val Loss: 1.6181
Epoch: 50/50... Step: 9010... Loss: 1.5286... Val Loss: 1.6149
Valid metrics improved (1.616142 --> 1.614902).  Saving model ...
Epoch: 50/50... Step: 9020... Loss: 1.4991... Val Loss: 1.6158
Epoch: 50/50... Step: 9030... Loss: 1.5360... Val Loss: 1.6179
Epoch: 50/50... Step: 9040... Loss: 1.4784... Val Loss: 1.6157
Epoch: 50/50... Step: 9050... Loss: 1.5776... Val Loss: 1.6187


In [20]:
# change the name, for saving multiple files
# model_name = 'text_gen.net'

# checkpoint = {'n_hidden': net.n_hidden,
#               'n_layers': net.n_layers,
#               'state_dict': net.state_dict(),
#               'tokens': net.chars}

# with open(model_name, 'wb') as f:
#     torch.save(checkpoint, f)

In [21]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [22]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [23]:
# untrained

In [30]:
print(sample(net, 1000, prime='get', top_k=5))

get into the
south and to a sear of story at
the thirders. The crust of it was
alported to the the first people
and a controls of toos
the troops, traits of
the strance, which
such as children of a
right, states, the topper,
with the centre, which also be a
rather is for the those of
their space on impacts with a stone
and animals to the extrande
telephine. He spaced
in the state and archaeology
to a sear of the consumms
to a speed to be such as
story in the potential
space. The power that a
control and continents
and telescope. They saped the
stal and structures as its still could
and a familiar animals to are
surplate species in the planet,
and the family of the stals.
Seen in the solar state
to the cell of the space
of the sea of the towns
in the second in spreads to the
fertile. The carad that
a single survone of the strength
to the engraves. They called
shift to bone to the
form of most to the tite
on the presence of monations
to but the size of the tomb
to colonies to support out

## Loading a checkpoint

In [28]:
# Here we have loaded in a model that trained
with open('text_gen.net', 'rb') as f:
    checkpoint = torch.load(f)
    
net = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
net.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [34]:
# trained
print(sample(net, 2000, top_k=5, prime="usually"))

usually.
In the end of the Universe can
seed this solerie to these control to suppress
which the earliest also settled.
1900
1785 1989
1906
1989
1969–2016
1989
1989 1959
2008 1000
2000
210
195
198
The corsum of Alexander when the
sprord for the farming. The soust and
allaric antelonist the contain of is illopitheted
and are steam of the terrical states of the
spanese and the surfece and an expleimed by called
carbon transformed from the since found, battles of
a plone. His transperies in a streed contricted by the state
of the energy and army, when into the streegs to be a
more to the settlement, allowing the south, by the complex
to animal a read star to called the comes and start to such
stars of the social common change and the set between
the sense of the first strong states their state of the
continent as in when the first stricks on the today was a
cloud of the transparation. The some of
the each oftical and state, becomes the
force in the proved in the standers, it was collaped 